### Import LSTM Model

In [4]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,KFold,cross_val_score as CVS
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE,mean_absolute_error as MAE,max_error as ME,r2_score as r2
import math

#import raw data
y_val = pd.read_csv('train_data_111.csv')
y_val['date']=pd.to_datetime(y_val['date'])
#data process
y_val.loc[y_val['LV1211 S01 PN111_BV25']>=65,'LV1211 S01 PN111_BV25']=np.nan
y_val['LV1211 S01 PN111_BV25']=y_val['LV1211 S01 PN111_BV25'].interpolate()

#interpolation
y_val.loc[y_val['LV1281 AS361 M1']==0,'LV1281 AS361 M1']=np.nan
y_val['LV1281 AS361 M1']=y_val['LV1281 AS361 M1'].interpolate()
y_val.loc[y_val['LV1281 VA307 BB42']==0,'LV1281 VA307 BB42']=np.nan
y_val['LV1281 VA307 BB42']=y_val['LV1281 VA307 BB42'].interpolate()
y_val.loc[y_val['LV1281 VZ366 BB40']==0,'LV1281 VZ366 BB40']=np.nan
y_val['LV1281 VZ366 BB40']=y_val['LV1281 VZ366 BB40'].interpolate()
y_val.loc[y_val['LV1281 WK363 BB40-1']==0,'LV1281 WK363 BB40-1']=np.nan
y_val['LV1281 WK363 BB40-1']=y_val['LV1281 WK363 BB40-1'].interpolate()
y_val.loc[y_val['LV1281 WK363 BB40-2']==0,'LV1281 WK363 BB40-2']=np.nan
y_val['LV1281 WK363 BB40-2']=y_val['LV1281 WK363 BB40-2'].interpolate()

tihuan = ['LV1281 S05 WU301 BB40','LV1281 S05 WU301 BB40-1','LV1281 S06 WU302 BB40','LV1281 S07 WU311 BB40','LV1281 S08 WU312 BB40','LV1281 S09 WU321 BB40']
for item in tihuan:
    y_val.loc[y_val[item] == 0, item] = np.nan
    y_val[item] = y_val[item].interpolate()

tihuan2=['LV1281 S07 WU311 M2','LV1281 S08 WU312 M2','LV1281 S09 WU321 M2','LV1281 S05 WU301 M2','LV1281  S06 WU302 M2']
y_val.loc[73149:73541, tihuan2] = np.nan
for item in tihuan2:
    y_val[item] = y_val[item].interpolate()

y_val.loc[29118:29133,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[37679:37698,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[57740:57755,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[70569:70585,['Oven1_NMHC','Oven1_NOx']] = np.nan
y_val.loc[78940:78954,['Oven1_NMHC','Oven1_NOx']] = np.nan

y_val.loc[24098:24102,['Oven1_NMHC']] = np.nan
y_val.loc[24118:24122,['Oven1_NMHC']] = np.nan
y_val.loc[74000:74004,['Oven1_NMHC']] = np.nan
y_val['Oven1_NMHC']=y_val['Oven1_NMHC'].interpolate()
y_val['Oven1_NOx']=y_val['Oven1_NOx'].interpolate()

Y_val =y_val.iloc[56886:81076,:].copy()
Y_val1 = y_val.iloc[20268:44076,:].copy()

# Difference for the linear growth variable
dif_col=['LV1281 S04 WR706 BV25-1','LV1281 S01 Energy consumption','LV1281 S02 Energy consumption','LV1281 S03 Energy consumption']
for item in dif_col:
    ts_diff = np.diff(Y_val[item])
    Y_val[item] = np.append([0], ts_diff)
    ts_diff = np.diff(Y_val1[item])
    Y_val1[item] = np.append([0], ts_diff)

# Two extracted time periods Y_val1 and Y_val1 are grouped together as the input data set
df1 = pd.concat([Y_val1, Y_val], ignore_index=True)
bxg=['LA1251 summary AmereMin charge','LV1211 S01 PN212_BV25','LV1211 S01 PN213_BV25','LV1281 AS361 M1','LV1281 VA307 M3','LV1281 WK363 BV25','LV1281 WK363 M2']
df1=df1.drop(columns=bxg)

# 17 variables selected and the current number of drying rooms
plot_variable=['LV1281 S04 WA506 BB40-5', 'LV1281 VZ366 BB40',
       'LV1281  S06 WU302 M2', 'LV1281 VA307 BB42',
        'LV1281 S04 WA506 BB40-2',
       'LV1281 S08 WU312 M2', 'LV1281 S04 WA506 BB40-4',
       'LV1211 S02 PR310_BN20', 'LV1281 S07 WU311 M2',
       'LV1281 S05 WU301 BB40-1','LV1211 S01 PN111_BL50',
       'LV1281 S09 WU321 M2',
       'LV1281 S05 WU301 M2', 'LV1211 S02 PR313_BB40-1',
        'LV1211 S01 PN111_BV25',
       'LV1211 S04 VZ503_M1_F', 'LV1211 S01 PR114_BV25',
       'LF1921 S01 TF270 Number of unit','Oven1_NMHC','Oven1_NOx']
df1=df1[plot_variable]
# Take the first 30 minutes and the first 90 to 60 minutes as input
def feature_engineer(df,names):
    N=30
    look_back=25
    skip = 60
    N2=30
    M=20
    lag_cols = [column for column in df][0:]
    shift_range = [x + 1 for x in range(N)]
    
    for i in shift_range:
        for col in lag_cols:
            new_col='{}_lag_{}'.format(col, i)   
            df[new_col]=df[col].shift(i)
    shift_range_y = [x + 1 for x in range(M)]
    for name in names:
        for i in shift_range_y: 
            new_col = '{}_pre_{}'.format(name, -i)
            df[new_col] = df[name].shift(-i)
    return df[look_back*skip:-M-1]
df = feature_engineer(df1,['Oven1_NMHC','Oven1_NOx'])
#2
continuous = [i for i in df.loc[:,df.nunique()>2]][0:]
categorical = [i for i in df.loc[:,df.nunique()<=2]]
#split train and test dataset
test_size = 0.3
num_test = int(test_size * len(df))
num_train = len(df)-num_test
train = df[:num_train]
test = df[num_train:]


scaler = StandardScaler().fit(train[continuous])
train_scaled = scaler.transform(train[continuous])
test_scaled = scaler.transform(test[continuous])

# Convert the numpy array back into pandas dataframe
train_scaled = pd.DataFrame(train_scaled, columns=continuous)
test_scaled = pd.DataFrame(test_scaled, columns=continuous)

# Obtain processed training set and test set data
train.reset_index(drop=True, inplace=True)
train_scaled.reset_index(drop=True, inplace=True)
temp_train = pd.concat([train[categorical],train_scaled],axis=1,join='inner')

test.reset_index(drop=True, inplace=True)
test_scaled.reset_index(drop=True, inplace=True)
temp_test = pd.concat([test[categorical],test_scaled],axis=1,join='inner')

X_train_scaled = temp_train.iloc[:,0:-40]
Y_train_scaled = temp_train.iloc[:,-40:]
X_test_scaled = temp_test.iloc[:,0:-40]
Y_test_scaled = temp_test.iloc[:,-40:]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
scaler_minmax1 = MinMaxScaler().fit(train[continuous].iloc[:,0:-40])
scaler_std1= StandardScaler().fit(train[continuous].iloc[:,0:-40])

scaler_std2 = StandardScaler().fit(train[continuous].iloc[:,-40:])

In [6]:
train_observation=train.iloc[:,0:620]
train_observation=train_observation.values
observation_high=train_observation.max(axis=0)
observation_low=train_observation.min(axis=0)

### Train deep rainforcement learning model

In [7]:
from keras.models import load_model

model_NMHC = load_model('lstm_model_NMHC.h5')
model_NOx = load_model('lstm_model_NOx.h5')
data_mean=[0.71,42.8]
data_std=[0.43,10.5]

Define objective function

In [8]:
class ObjectiveFunction(object):
    @staticmethod
    def emission(X):
        X=np.array(X)

#         X=scaler_minmax1.transform(X.reshape(1, -1))
        X=scaler_std1.transform(X.reshape(1, -1))#normalize
        train_scaled = np.reshape(X, (X.shape[0], 1, X.shape[1]))
        y_NMHC=model_NMHC.predict(train_scaled)[0]
        y_NOx=model_NOx.predict(train_scaled)[0]
        #Disnormalize the resulting predicted value
        y_std= np.append(y_NMHC,y_NOx, axis=0)
        y=scaler_std2.inverse_transform(y_std.reshape(1, -1))
        return y[0]#returned the amount of  pollutants emitted over the next 20 minutes

    @staticmethod
    # N is the number of vehicles in the drying room
    def weighted_sum(y,N):#传入scaler_std2
        # Rewards associated with NOx emissions
        y_NMHC=np.mean(y[0:20])
        y_NOx=np.mean(y[20:])
        if y_NOx<data_mean[1]:
            if N==0:
                reward1 = (N+1)/(data_std[1]/(data_mean[1]-y_NOx));
                reward1 =reward1*100;
            else:
                reward1 = N/(data_std[1]/(data_mean[1]-y_NOx));
                reward1 =reward1*100;
        else:
            if N==0:
                reward1 = -(1000/(N+1))*(1/(data_std[1]/(y_NOx-data_mean[1])));
                reward1 =reward1*100;
            else:
                reward1 = -(1000/N)*(1/(data_std[1]/(y_NOx-data_mean[1])));
                reward1 =reward1*100;  

        #Rewards associated with NMHC emissions
        if y_NMHC<data_mean[0]:
            if N==0:
                reward2 = (N+1)/(data_std[0]/(data_mean[0]-y_NMHC));
                reward2 =reward2*100;
            else:
                reward2 = N/(data_std[0]/(data_mean[0]-y_NMHC));
                reward2 =reward2*100;
        else:
            if N==0:
                reward2 = -(100/(N+1))*(1/(data_std[0]/(y_NMHC-data_mean[0])));
                reward2 =reward2*100;
            else:
                reward2 = -(100/N)*(1/(data_std[0]/(y_NMHC-data_mean[0])));
                reward2 =reward2*100;
        # Do not exceed the limits of prescribed pollutant discharge
        if y_NOx>123 or y_NMHC>1.75:
            reward3=-100;
        else:
            reward3=0

        reward=reward1+reward2+reward3
        return reward

DDPG defination

In [14]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--env_name', default='RLWWTP-v0')
parser.add_argument('--tau', default=0.005, type=float)
parser.add_argument('--target_update_interval', default=1, type=int)
parser.add_argument('--learning_rate', default=1e-2, type=float)
parser.add_argument('--gamma', default=0.99, type=float)
parser.add_argument('--capacity', default=3000, type=int)
parser.add_argument('--capacity2', default=600, type=int)

parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--seed', default=False, type=bool)
parser.add_argument('--random_seed', default=10, type=int)

parser.add_argument('--log_interval', default=50, type=int)
parser.add_argument('--load', default=False, type=bool)
parser.add_argument('--max_episode', default=100, type=int)
parser.add_argument('--max_length_of_trajectory', default=1200, type=int)  # length of each trajectory,1200是期望
parser.add_argument('--print_log', default=1, type=int)
parser.add_argument('--update_iteration', default=10, type=int)
args = parser.parse_args(args=[])

In [11]:
!pip install tensorboardX

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from tensorboardX import SummaryWriter

args = parser.parse_args([])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()


class ReplayBuffer:
    def __init__(self, max_size=args.capacity):
        self.storage = []
        self.max_size = max_size
        self.ptr = 0

    def push(self, data):
        if len(self.storage) == self.max_size:  # if buffer is full, cover old data.
            self.storage[int(self.ptr)] = data
            self.ptr = (self.ptr + 1) % self.max_size
        else:
            self.storage.append(data)

    def sample(self, batch_size):
        ind = np.random.randint(0, len(self.storage), size=batch_size)
        state_buffer, next_state_buffer, action_buffer, reward_buffer, done_buffer = [], [], [], [], []

        for i in ind:
            state, next_state, action, reward, done = self.storage[i]
            state_buffer.append(np.array(state, copy=False))
            next_state_buffer.append(np.array(next_state, copy=False))
            action_buffer.append(np.array(action, copy=False))
            reward_buffer.append(np.array(reward, copy=False))
            done_buffer.append(np.array(done, copy=False))

        return np.array(state_buffer), np.array(next_state_buffer), np.array(action_buffer), \
               np.array(reward_buffer).reshape(-1, 1), np.array(done_buffer).reshape(-1, 1)


class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action, min_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 64)
        self.l2 = nn.Linear(64, 128)
        self.l3 = nn.Linear(128, 64)
        self.l4 = nn.Linear(64, action_dim)

        self.max_action = torch.from_numpy(max_action).to(device)
        self.min_action = torch.from_numpy(min_action).to(device)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        #x = self.max_action * torch.sigmoid(self.l4(x))
        x = torch.sigmoid(self.l4(x))
        return x


class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.l1 = nn.Linear(state_dim + action_dim, 64)
        self.l2 = nn.Linear(64, 128)
        self.l3 = nn.Linear(128, 64)
        self.l4 = nn.Linear(64, 1)

    def forward(self, x, u):
#         a = torch.cat([x, u], 1).unsqueeze(0)
#         x = F.relu(self.l1(a))
        x = F.relu(self.l1(torch.cat([x, u], 1)))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = self.l4(x)
        return x


class DDPG(object):
    def __init__(self, state_dim, action_dim, max_action, min_action, phase=1):
        if phase != 1:
            self.actor = torch.load('./model/actor.pkl', map_location='cuda')
            self.actor.to(device)
        elif phase == 1:
            self.actor = Actor(state_dim, action_dim, max_action, min_action).to(device)
        self.actor_target = Actor(state_dim, action_dim, max_action, min_action).to(device)

        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(),args.learning_rate)

        if phase != 1:
            self.critic = torch.load('./model/critic.pkl', map_location='cuda')
            self.critic.to(device)
        elif phase == 1:
            self.critic = Critic(state_dim, action_dim).to(device)
        self.critic_target = Critic(state_dim, action_dim).to(device)
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), args.learning_rate)

        self.replay_buffer = ReplayBuffer()
        self.writer = SummaryWriter('./log/')
        self.num_critic_update_iteration = 0
        self.num_actor_update_iteration = 0
        self.num_training = 0

    def selection_action(self, state):
        # state = torch.FloatTensor(np.array(state).reshape(1,-1)).to(device)
        state = torch.FloatTensor(np.array(state)).to(device)
        return self.actor(state).cpu().detach().numpy()

    def update(self, agents):
        for it in range(args.update_iteration):
            state, next_state, action, reward, done = self.replay_buffer.sample(args.batch_size)
            state = torch.FloatTensor(state).to(device)
            # state = torch.FloatTensor(state).to(device)
            # action = torch.FloatTensor(action).squeeze(12).to(device)
            action = torch.FloatTensor(action).to(device)
            # next_state = torch.FloatTensor(next_state.reshape(-1,1)).to(device)
            next_state = torch.FloatTensor(next_state).to(device)
            done = torch.FloatTensor(done).to(device)
            reward = torch.FloatTensor(reward).to(device)

            #update target_network
            #Use target actor to get the corresponding output based on the corresponding input
            # x0=agents.actor_target(next_state)

            # target_action = x0.float()
            x0=self.actor_target(next_state)
            target_Q = self.critic_target(next_state, self.actor_target(next_state))
            target_Q = reward + ((1 - done) * args.gamma * target_Q).detach()
            current_Q = self.critic(state, action)

            # update critic network
            critic_loss = F.mse_loss(current_Q, target_Q)
            self.writer.add_scalar('Loss/critic_loss', critic_loss, global_step=self.num_critic_update_iteration)
            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()
            # update actor network
            # x1=agents.actor(state)

            # cur_action = x1.float()
            # actor_loss = -self.critic(state, cur_action).mean()
            actor_loss = -self.critic(state, self.actor(state)).mean()
            self.writer.add_scalar('Loss/actor_loss', actor_loss, global_step=self.num_actor_update_iteration)
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # update target network
            for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)
            for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(args.tau * param.data + (1 - args.tau) * target_param.data)

            self.num_actor_update_iteration += 1
            self.num_critic_update_iteration += 1
            self.writer.close()

environment

In [17]:
import logging
import math
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import time
from utils.tool import clear_text,txt_read
import torch
import pandas as pd
import random

logger = logging.getLogger(__name__)


class RLWWTP(gym.Env):
    
    metadata = {'render.modes': ['human', 'rgb_array'],'video.frames_per_second': 50 }
  
    def __init__(self):
        # state parameters
        self.state_size=620
        self.observation_high = observation_high
        self.observation_low = observation_low

        #action parameters
        self.action_size=17
        self.action_high = np.array([475,32.37,100,54,164,36,635,97.3,100,310,1000,50,35,43,61.8,44,72])
        self.action_low = np.array([376,9.67,60,18,152,9,580,92,75,200,600,20,0,38,59.7,44,0])
        
        self.action_high2 = np.array([150,36,1,43,100,1,200,96,1,150,856,1,1,43,61.7,45,0.029])
        self.action_low2 = np.array([50,18,0,25,31,0,55,0,0,40,466,0,0,26,60.9,0,0])
        
        # The action space is determined based on the maximum and minimum values of the defined action
        # Two action Spaces are set respectively for two agents
        self.action_space = spaces.Box(low=self.action_low, high=self.action_high, dtype=np.float32)#spaces.Box创造动作的连续区间
        self.action_space2 = spaces.Box(low=self.action_low2, high=self.action_high2, dtype=np.float32)#spaces.Box创造动作的连续区间
        # define state space
        self.observation_space = spaces.Box(low=self.observation_low, high=self.observation_high, dtype=np.float32)#spaces.Box创造observation的连续空间
        # Initialize the environment state
        self.state_array=np.array(train[continuous].iloc[5300,0:-40])
        self.phase_var=0
        self.is_low1=False
        self.is_low2=False
        self.is_low3=False
        self.is_shake1=True
        self.is_shake2=True
        self.is_shake3=True
        self.count=0
        self.amount=0
        self.seed()
#     self.reset()
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, actions,y,count,current_car):

        ################################### According to the current number of vehicles, determine which agent selected the action, and the corresponding anti-normalization
        actions=actions
        if current_car>5:
            for i in range(len(actions)):
                actions[i]=self.action_low[i]+actions[i]*(self.action_high[i]-self.action_low[i])         
        else:
            for i in range(len(actions)):
                actions[i]=self.action_low2[i]+actions[i]*(self.action_high2[i]-self.action_low2[i])
                
        #np.savetxt(action_path_1)
        # Five environmental states, phase_var=0 corresponds to normal production; phase_var=1 corresponds to production repair state 1; phase_var=2 corresponds to production refurbishment state 2; phase_var=3 corresponds to production repair state 3;
        # phase_var=4 corresponds to the shutdown process. phase_var=5 indicates that production is resumed after shutdown
        # 0: indicates the environment change during normal production
        rand1=random.uniform(0,1)
        if self.phase_var==0:
            # rand1<0.006, Enter the production finishing section
            if rand1<0.006:
                self.count+=1
                # First three rounds of production refurbishment before entering the shutdown
                if self.count<4:
                    p=np.array([1/3,1/3,1/3])
                    self.phase_var=np.random.choice([1,2,3],p=p.ravel())# Randomly enter one of the production repair states
                else:
                    self.phase_var=4;

            else:
            # 
                if self.state==21:
                  # Choose with a certain probability 1，0，-1
                    p=np.array([0.05,0.9,0.05])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                # If st is not equal to 21, return to 21
                elif self.state<21:self.state+=1;
                else :self.state-=1;
        # 1: Environment change in repair state 1
        elif self.phase_var==1:
          # First descent
            if self.is_shake1==True:
            # Has not yet reached the lowest point 9
                if self.state>9 and self.is_low1==False:
                    p=np.array([1/15,1/3,0.6])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                    # Determine if the bottom has been reached
                    if self.state==9:
                        self.is_low1=True;   
                elif self.is_low1==True and self.state==9:
                    # The number of vehicles oscillates around 9, generates a random number, and jumps out of the oscillating process with a certain probability
                    p=np.array([0.25,0.5,0.25])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                    rand2=random.uniform(0,1)
                    if rand2<=0.1:
                        self.is_shake1=False;# downward oscillation, and the number of vehicles begins to increase

                else:
                  # the number of vehicles return to 9
                    if self.state<9: self.state+=1;
                    else:self.state-=1;
                    rand3=random.uniform(0,1)
                    if rand3<=0.1:
                        self.is_shake1=False

            else:
                # The number of cars has not reached 21
                if self.state<21:
                    p=np.array([0.6,1/3,1/15])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())

                else:
                    self.phase_var=0# Return to normal production
                    self.is_shake1=True;# Restore parameters to default
                    self.is_low1=False

        elif self.phase_var==2:# Production refurbishment state 2 Changes in the number of bodies
            if self.is_shake2==True:
                if self.state>5 and self.is_low2==False:
                    p=np.array([1/15,1/3,0.6])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                    if self.state==5:
                        self.is_low2=True;

                elif self.is_low2==True and self.state==5:
                    p=np.array([0.25,0.5,0.25])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                    rand2=random.uniform(0,1)
                    if rand2<=0.1:
                        self.is_shake2=False;

                else:
                    if self.state<5: self.state+=1;
                    else:self.state-=1;
                    rand3=random.uniform(0,1)
                    if rand3<=0.1:
                        self.is_shake2=False

            else:
                if self.state<21:
                    p=np.array([0.6,1/3,1/15])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                else:
                    self.phase_var=0
                    self.is_shake2=True;
                    self.is_low2=False

        elif self.phase_var==3:        
            if self.is_shake3==True:
                if self.state>15 and self.is_low3==False:
                    p=np.array([1/15,1/3,0.6])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                    if self.state==15:
                        self.is_low3=True;

                elif self.is_low3==True and self.state==15:
                    p=np.array([0.25,0.5,0.25])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())
                    rand2=random.uniform(0,1)
                    if rand2<=0.1:
                        self.is_shake3=False;

                else:
                    if self.state<15: self.state+=1;
                    else:self.state-=1;
                    rand3=random.uniform(0,1)
                    if rand3<=0.1:
                        self.is_shake3=False

            else:
                if self.state<21:
                    p=np.array([0.6,1/3,1/15])
                    self.state+=np.random.choice([1,0,-1],p=p.ravel())

                else:
                    self.phase_var=0
                    self.is_shake3=True;
                    self.is_low3=False
        elif self.phase_var==4:
            if self.state>0:
                p=np.array([1/15,1/3,0.6])
                self.state+=np.random.choice([1,0,-1],p=p.ravel())

            else:
                rand4=random.uniform(0,1)
                if rand4<0.002:
                    self.phase_var=5

        else:
            if self.state<21:
                p=np.array([0.6,1/3,1/15])
                self.state+=np.random.choice([1,0,-1],p=p.ravel()) 
            else:
                self.phase_var=0
                
        state=self.state
        actions_tmp=list(actions)
        actions_tmp.append(state)
        actions_tmp.append(y[0][0])
        actions_tmp.append(y[0][20])
        # Update the status
        self.state_array= np.append(actions_tmp,self.state_array, axis=0)
        self.state_array=self.state_array[:len(self.state_array)-20]
        # Check whether the changed state exceeds the boundary
        for i in range(len(self.state_array)):
            if self.state_array[i] < self.observation_low[i]: self.state_array[i] = self.observation_low[i]
            if self.state_array[i] > self.observation_high[i]: self.state_array[i] = self.observation_high[i]
                
        reward_1 = self.reward_calculate(self.state_array)
        
        ddpg_selection_result={"State":self.state,"Reward":reward_1,"LV1281 S04 WA506 BB40-5":actions_tmp[0],'LV1281 VZ366 BB40':actions_tmp[1],
                                 'LV1281  S06 WU302 M2':actions_tmp[2],'LV1281 VA307 BB42':actions_tmp[3],
                                  'LV1281 S04 WA506 BB40-2':actions_tmp[4],'LV1281 S08 WU312 M2':actions_tmp[5],
                                  'LV1281 S04 WA506 BB40-4':actions_tmp[6],'LV1211 S02 PR310_BN20':actions_tmp[7],
                                  'LV1281 S07 WU311 M2':actions_tmp[8],'LV1281 S05 WU301 BB40-1':actions_tmp[9],
                                  'LV1211 S01 PN111_BL50':actions_tmp[10],'LV1281 S09 WU321 M2':actions_tmp[11],
                                  'LV1281 S05 WU301 M2':actions_tmp[12],'LV1211 S02 PR313_BB40-1':actions_tmp[13],
                                  'LV1211 S01 PN111_BV25':actions_tmp[14],'LV1211 S04 VZ503_M1_F':actions_tmp[15],
                                  'LV1211 S01 PR114_BV25':actions_tmp[16],'LF1921 S01 TF270 Number of unit':actions_tmp[17],
                                  'Oven1_NMHC':actions_tmp[18],'Oven1_NOx':actions_tmp[19]}

        df=pd.DataFrame(data=ddpg_selection_result,index=[count])
        df.to_csv("ddpgResult7.csv", encoding="utf-8", mode="a", header=False)
        return self.state_array, reward_1, False, {}
    def reward_calculate(self,X):
        y=ObjectiveFunction.emission(X)
        reward=ObjectiveFunction.weighted_sum(y,self.state)
        return reward

    def reset(self):
        # Game initialization
        self.state=random.randint(20, 22)
        self.phase_var =0
        self.is_low1=False
        self.is_low2=False
        self.is_low3=False
        self.is_shake1=True
        self.is_shake2=True
        self.is_shake3=True
        self.count=0;
        self.amount=0
        state = self.state
        self.state_array=np.array(train[continuous].iloc[5300,0:-40])
        return self.state_array

Parameters of MADDPG：  
state_dim, action_dim, max_action, min_action, actor_path='', critic_path=''，phase=1, actor_lr=1e-3, critic_lr=1e-3)

In [ ]:
import os
import gym
import numpy as np
import torch
from utils.logger import save_output, concatenate_data
from utils.plot import action_plot, reward_plot

print(torch.__version__)
args = parser.parse_args([])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

env = RLWWTP();

# random seed generation
if args.seed:
    env.seed(args.random_seed)
    torch.manual_seed(args.random_seed)
    np.random.seed(args.random_seed)

# basic info
state_dim = 620
action_dim = 17
max_action = env.action_space.high
min_action = env.action_space.low

max_action_dosage = env.action_space2.high
min_action_dosage = env.action_space2.low

min_val = torch.tensor(1e-7).float().to(device)

def main(i, transfer=False):
    count=0
    episode_reward = []
    episode_aer = []
    lr = args.learning_rate
    agent_do = DDPG(state_dim, action_dim, max_action, min_action, phase=1,)  # initialize ddpg
    agent_dosage = DDPG(state_dim, action_dim, max_action_dosage, min_action_dosage, phase=1,)  # initialize ddpg
    ep_r = 0
    ep_a = []
    if transfer:
        #agent_do
        for name, param in agent_do.actor.named_parameters():
            if name.startswith('l4') or name.startswith('l3') or name.startswith('l2'):
                param.requires_grad = True
            else:
                param.requires_grad = False

        for name, param in agent_do.critic.named_parameters():
            if name.startswith('l4') or name.startswith('l3') or name.startswith('l2'):
                param.requires_grad = True
            else:
                param.requires_grad = False
        #agent_dosage       
        for name, param in agent_dosage.actor.named_parameters():
            if name.startswith('l4') or name.startswith('l3') or name.startswith('l2'):
                param.requires_grad = True
            else:
                param.requires_grad = False

        for name, param in agent_dosage.critic.named_parameters():
            if name.startswith('l4') or name.startswith('l3') or name.startswith('l2'):
                param.requires_grad = True
            else:
                param.requires_grad = False
                
    for epoch in range(args.max_episode):
        state_do = env.reset()
        t=0
        while True:
            count+=1
            # determine which agent to take to select the action according to the current number of drying vehicles
            if state_do[17]<=5:
                action_do_1 = agent_dosage.selection_action(state_do)  # generate actions, where state is the input and action is the output
            else:
                action_do_1 = agent_do.selection_action(state_do)  # generate actions, where state is the input and action is the output
                
            if transfer:
                noise = max(0.2 - 0.01 * epoch, 0.0)
            else:
                noise = max(0.2 - 0.0004 * epoch, 0.0)

            action_do_1 = (action_do_1 + np.random.normal(0, noise, size=17)).clip(0.0, 1.0) # add noise for exploration. Keep it between 0 and 1
            # transition
            state_do=np.array(state_do)
            X=scaler_std1.transform(state_do.reshape(1, -1))# Normalize the values
            train_scaled = np.reshape(X, (X.shape[0], 1, X.shape[1]))
            # Get the emission of pollutants at the current moment
            y_NMHC=model_NMHC.predict(train_scaled)[0]
            y_NOx=model_NOx.predict(train_scaled)[0]
            
            y_std= np.append(y_NMHC,y_NOx, axis=0)# The two pollutants are discharged together
            y=scaler_std2.inverse_transform(y_std.reshape(1, -1))
            next_state_1, reward_1, done_1, info_1 = env.step(action_do_1,y,count,current_car=state_do[17])
            next_state_do = next_state_1
            
            ep_r += reward_1
            ##### determine which agent's buffer to add data to, based on the actual number of vehicles
            if state_do[17]<=5:
                agent_dosage.replay_buffer.push(
                    (state_do, next_state_do,action_do_1,reward_1, np.float(done_1)))
            else:
                agent_do.replay_buffer.push(
                    (state_do, next_state_do,action_do_1,reward_1, np.float(done_1)))                

            state_do = next_state_do
            t += 1
            if t >= args.max_length_of_trajectory:# Is this trajectory a total of how many movements, more than you can stop
                if epoch % args.print_log == 0:
                    print("Ep_i \t{}, the ep_r is \t{:0.2f}, the step is \t{}".format(epoch, ep_r / t, t))
                episode_reward.append(ep_r / t)
                ep_r = 0
                ep_a=[]
                save_output(episode_reward=episode_reward, save_path='reward/ddpg_final{}.txt'.format(i, epoch))
                break

        # update network
        if len(agent_do.replay_buffer.storage) >= args.capacity - 1:
            agent_do.update(agent_do)      
            torch.save(agent_do.actor, 'model/actor_do.pkl')
            torch.save(agent_do.critic, 'model/critic_do.pkl')
            save_output(episode_reward=episode_reward, save_path='reward/ddpg_{}_{}.txt'.format(i, phase))
            #save_output(episode_reward=episode_aer, save_path='aer/aer_{}_{}.txt'.format(i, phase))
        
        # When the number of vehicles is small, the sample size is relatively small, so the two agents are updated separately
        if len(agent_dosage.replay_buffer.storage) >= args.capacity2 - 1:
            agent_dosage.update(agent_dosage)
            torch.save(agent_dosage.actor, 'model/actor_dosage.pkl')
            torch.save(agent_dosage.critic, 'model/critic_dosage.pkl')
        
if __name__ == '__main__':
    num = 1
    phase = 2
    for i in range(num):
        print('Recursion {}'.format(i))
        main(i)

    #df_reward = concatenate_data('reward/', num=num*args.max_episode, algorithm='DDPG')
    #df_aer = concatenate_data('aer/', num=num*args.max_episode, algorithm='aer')
    #reward_plot(data=df_reward, values=['DDPG'], ylabel='Reward')

1.8.1+cu101
Recursion 0


/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Ep_i 	0, the ep_r is 	-2302.42, the step is 	1200
Ep_i 	1, the ep_r is 	3297.62, the step is 	1200
Ep_i 	2, the ep_r is 	-3238.31, the step is 	1200
Ep_i 	3, the ep_r is 	4189.64, the step is 	1200
Ep_i 	4, the ep_r is 	2770.67, the step is 	1200
Ep_i 	5, the ep_r is 	4479.95, the step is 	1200
Ep_i 	6, the ep_r is 	2756.23, the step is 	1200
Ep_i 	7, the ep_r is 	2822.71, the step is 	1200
Ep_i 	8, the ep_r is 	4410.66, the step is 	1200
Ep_i 	9, the ep_r is 	2053.35, the step is 	1200
Ep_i 	10, the ep_r is 	4237.50, the step is 	1200
Ep_i 	11, the ep_r is 	1985.84, the step is 	1200
Ep_i 	12, the ep_r is 	3292.33, the step is 	1200
Ep_i 	13, the ep_r is 	3445.27, the step is 	1200
Ep_i 	14, the ep_r is 	3224.19, the step is 	1200
Ep_i 	15, the ep_r is 	1827.13, the step is 	1200
Ep_i 	16, the ep_r is 	1316.68, the step is 	1200
Ep_i 	17, the ep_r is 	1431.95, the step is 	1200
Ep_i 	18, the ep_r is 	3855.34, the step is 	1200
Ep_i 	19, the ep_r is 	3293.80, the step is 	1200
Ep_i 	20

In [ ]:
hhhhhhhhhhhhhh
jjjj
kkk